In [265]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import fiona
import glob
from shapely.geometry import Point

from methods import read_shst_extract

In [2]:
data_interim_dir = "../../data/interim/"

In [50]:
link_file = data_interim_dir + "step3_join_shst_extraction_with_osm/" + "link.json"
with open(link_file) as f:
    link_json = json.load(f)
link_df = pd.DataFrame(link_json)

shape_gdf = gpd.read_file(data_interim_dir + "step5_tidy_roadway/" 
                          + "shape.geojson")

link_df = pd.merge(link_df,
                   shape_gdf[["id", "geometry"]],
                   how = "left",
                   left_on = "shstGeometryId",
                   right_on = "id")

link_df = gpd.GeoDataFrame(link_df, geometry = link_df["geometry"],
                            crs={"init" : "epsg:4326"})

In [51]:
pems_file = "../../data/external/mtc/pems_period.csv"

pems_df = pd.read_csv(pems_file)

In [52]:
pems_df["geometry"] = [Point(xy) for xy in zip(pems_df.longitude, pems_df.latitude)]

pems_gdf = gpd.GeoDataFrame(pems_df, geometry = pems_df["geometry"],
                            crs={"init" : "epsg:4326"})

pems_gdf = pems_gdf[~((pems_gdf.longitude.isnull()) | (pems_gdf.latitude.isnull()))]

# Write out pems for shst conflation

In [ ]:
# keep unique

pems_gdf.drop_duplicates(subset = ["station", "longitude", "latitude"])[["station", "longitude", "latitude", "geometry"]].to_file("../../data/external/mtc/pems.in.geojson",
                                                driver = "GeoJSON")

# Prepare for nearest match

In [53]:
# convert crs to meter based

pems_gdf = pems_gdf.to_crs(epsg = 26915)

link_df = link_df.to_crs(epsg = 26915)

In [27]:
pems_gdf.shape

(76057, 23)

In [28]:
pems_gdf.route.value_counts().sum()

76057

In [11]:
link_df.sindex

In [81]:
pems_gdf.iloc[2]

station                                               400000
district                                                   4
route                                                    101
direction                                                  S
type                                                      ML
time_period                                               EV
lanes                                                      3
median_flow                                             8523
avg_flow                                             8434.26
sd_flow                                              420.136
median_speed                                         67.3375
avg_speed                                            67.2283
sd_speed                                             0.69445
median_occup                                        0.026681
avg_occup                                           0.026709
sd_occupancy                                      0.00136179
days_observed           

In [22]:
link_df[link_df.tomtom_shieldnum == "101"].tomtom_rtedir

4970       S
6024       N
6125       S
6956       S
6968       S
8399        
9141       N
9512       N
9548        
10390      N
10523      S
12406      S
14202       
15208      S
15489      S
15550      N
17420      N
17509       
18462      N
20015       
21190      S
22408      S
23768      S
26301      S
27759      S
28869      S
30642       
31392       
31457      N
31813       
          ..
877979     N
878106     S
878677     N
879254      
879674     S
880469      
880972     S
882123     N
882162     N
882172     S
883411     N
884964     S
886680     S
886991     N
887567     S
887754     S
892888      
893695     S
895945     N
897309     N
897710     S
898910     N
900122     S
901751      
903199     N
903902     N
906160     S
906462     N
1293897    S
1534868     
Name: tomtom_rtedir, Length: 806, dtype: object

# Write out links that have shieldnum same as pems route

In [98]:
[c for c in pems_gdf.route.unique().astype(str) if c not in link_df.tomtom_shieldnum.unique()]

['948']

In [96]:
link_df.tomtom_shieldnum.unique()
pems_gdf.route.unique().astype(str)

array(['101', '80', '680', '280', '880', '580', '24', '4', '238', '85',
       '87', '17', '237', '242', '92', '1', '980', '25', '37', '948',
       '780', '84', '29', '156', '205', '380', '12', '152', '160'],
      dtype='<U21')

In [99]:
interest_facility_df = link_df[link_df.tomtom_shieldnum.isin(pems_gdf.route.unique().astype(str))]

In [101]:
interest_facility_df.to_file(data_interim_dir + "mtc/link_candidates_for_pems.geojson",
                            driver = "GeoJSON")

In [102]:
interest_facility_df.groupby(["tomtom_shieldnum", "tomtom_rtedir"]).count()

access  area  bike_access  bridge  \
tomtom_shieldnum tomtom_rtedir                                      
1                                 1205  1205         1205    1205   
                 N                 121   121          121     121   
                 S                 140   140          140     140   
101                                205   205          205     205   
                 N                 284   284          284     284   
                 S                 317   317          317     317   
12                                1007  1007         1007    1007   
                 E                  17    17           17      17   
                 W                  18    18           18      18   
152                                388   388          388     388   
156                                  4     4            4       4   
160                                  2     2            2       2   
                 N                   4     4            4       4   
                 S                   3     3            3       3   
17                                  27    27           27      27   
                 N                  17    17           17      17   
                 S                  17    17           17      17   
205              E                   1     1            1       1   
                 W                   2     2            2       2   
237                                 82    82           82      82   
                 E                  20    20           20      20   
                 W                  20    20           20      20   
238                                498   498          498     498   
                 N                  10    10           10      10   
                 S                   6     6            6       6   
24               E                  31    31           31      31   
                 W                  29    29           29      29   
242              N                   7     7            7       7   
                 S                   8     8            8       8   
25                                  20    20           20      20   
...                                ...   ...          ...     ...   
37               E                  13    13           13      13   
                 W                  21    21           21      21   
380              E                   5     5            5       5   
                 W                   4     4            4       4   
4                                   96    96           96      96   
                 E                  57    57           57      57   
                 W                  57    57           57      57   
580              E                 134   134          134     134   
                 W                 118   118          118     118   
680              N                 110   110          110     110   
                 S                 108   108          108     108   
780              E                  19    19           19      19   
                 W                  19    19           19      19   
80               E                 127   127          127     127   
                 W                 145   145          145     145   
84                                 818   818          818     818   
                 E                  12    12           12      12   
                 W                  19    19           19      19   
85               N                  56    56           56      56   
                 S                  46    46           46      46   
87                                  11    11           11      11   
                 N                  17    17           17      17   
                 S                  23    23           23      23   
880              N                 106   106          106     106   
                 S                 105   105          105     105   
92                            

In [103]:
pems_gdf.groupby(["route", "direction"]).count()

station  district  type  time_period  lanes  median_flow  \
route direction                                                             
1     N               35        35    35           35     35           35   
      S               30        30    30           30     30           30   
4     E             1756      1756  1756         1756   1756         1756   
      W             1800      1800  1800         1800   1800         1800   
12    E              110       110   110          110    110          110   
      W              100       100   100          100    100          100   
17    N              700       700   700          700    700          700   
      S              439       439   439          439    439          439   
24    E             1221      1221  1221         1221   1221         1221   
      W             1455      1455  1455         1455   1455         1455   
25    N              182       182   182          182    182          182   
      S              183       183   183          183    183          183   
29    N               65        65    65           65     65           65   
      S               75        75    75           75     75           75   
37    E              283       283   283          283    283          283   
      W              352       352   352          352    352          352   
80    E             5167      5167  5167         5167   5167         5167   
      W             5089      5089  5089         5089   5089         5089   
84    E              355       355   355          355    355          355   
      W              330       330   330          330    330          330   
85    N             1670      1670  1670         1670   1670         1670   
      S             1635      1635  1635         1635   1635         1635   
87    N              652       652   652          652    652          652   
      S              674       674   674          674    674          674   
92    E              727       727   727          727    727          727   
      W              729       729   729          729    729          729   
101   N             9914      9914  9914         9914   9914         9914   
      S             9344      9344  9344         9344   9344         9344   
152   E               30        30    30           30     30           30   
      W               25        25    25           25     25           25   
156   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
160   N               15        15    15           15     15           15   
      S               15        15    15           15     15           15   
205   E               15        15    15           15     15           15   
      W               19        19    19           19     19           19   
237   E              388       388   388          388    388          388   
      W              509       509   509          509    509          509   
238   N              291       291   291          291    291          291   
      S              299       299   299          299    299          299   
242   N              224       224   224          224    224          224   
      S              232       232   232          232    232          232   
280   N             1832      1832  1832         1832   1832         1832   
      S             1794      1794  1794         1794   1794         1794   
380   E               20        20    20           20     20           20   
      W               20        20    20           20     20           20   
580   E             2765      2765  2765         2765   2765         2765   
      W             2671      2671  2671         2671   2671         2671   
680   N             4358      4358  4358         4358   4358         4358   
      S             4687      4687  4687         4687   4687         4687   
780   E      

In [86]:
link_df.tomtom_shieldnum.value_counts()

        862208
nan     826373
82        2703
1         1466
12        1042
84         849
101        806
185        768
123        739
116        716
128        611
238        514
35         513
29         507
152        388
130        387
G4         317
121        294
G2         278
80         272
580        252
280        251
J2         251
61         242
92         235
680        218
113        214
880        211
4          210
13         209
         ...  
131        125
237        122
G3         106
85         102
112         88
260         77
2063        74
37          73
G8          64
10          62
17          61
24          60
109         59
87          51
780         38
221         29
G9          27
220         27
93          24
25          20
505         19
262         19
242         15
980         14
77          12
G7          12
160          9
380          9
156          4
205          3
Name: tomtom_shieldnum, Length: 62, dtype: int64

In [202]:
pems_gdf[(pems_gdf.route == 101) & (pems_gdf.direction == "S")]

,station,district,route,direction,type,time_period,lanes,median_flow,avg_flow,sd_flow,...,median_occup,avg_occup,sd_occupancy,days_observed,state_pm,abs_pm,latitude,longitude,year,geometry
0,400000,4,101,S,ML,AM,3,16383.0,16237.843137,691.559263,...,0.130709,0.141076,0.047421,51,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
1,400000,4,101,S,ML,EA,3,4366.5,4376.357143,102.251993,...,0.041496,0.041541,0.001666,56,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
2,400000,4,101,S,ML,EV,3,8523.0,8434.255814,420.135754,...,0.026681,0.026709,0.001362,43,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
3,400000,4,101,S,ML,MD,3,18178.0,18298.847826,560.210197,...,0.067676,0.068619,0.004611,46,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
4,400000,4,101,S,ML,PM,3,14616.5,14651.839286,471.933769,...,0.065185,0.065373,0.002153,56,288.0,459.520,38.081167,-122.547606,2005,POINT (-2115315.56035313 4652743.598747026)
66,400043,4,101,S,ML,AM,4,23577.0,23416.629630,748.084530,...,0.098076,0.103209,0.014871,54,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
67,400043,4,101,S,ML,EA,4,4928.0,4930.222222,173.380659,...,0.033719,0.033801,0.001366,63,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
68,400043,4,101,S,ML,EV,4,25855.5,25716.104167,1250.793778,...,0.067001,0.067381,0.003901,48,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
69,400043,4,101,S,ML,MD,4,28495.0,28507.380000,651.071264,...,0.095326,0.095746,0.002916,50,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)
70,400043,4,101,S,ML,PM,4,26542.0,26266.150000,914.049248,...,0.118074,0.126354,0.019513,60,445.0,431.631,37.753591,-122.403092,2005,POINT (-2115244.798598309 4610955.231681113)


# Match Pems to the nearest link that has same shieldnum and direction

In [238]:
# match based on pems route, direction, and tomtom shieldnum, rtedir

offset = 50

pems_route_list = pems_gdf.route.unique().tolist()

pems_match_gdf = gpd.GeoDataFrame()

for route in pems_route_list:
    pems_subset_gdf = pems_gdf[pems_gdf.route == route].copy()
    
    dir_list = pems_subset_gdf.direction.unique().tolist()
    
    print("pems route id {}".format(route))
    
    for direction in dir_list:
        
        print("\t pems direction {}".format(direction))
        
        pems_subset_gdf = pems_gdf[(pems_gdf.route == route) & (pems_gdf.direction == direction)].copy()
        
        bbox = pems_subset_gdf.bounds + [-offset, -offset, offset, offset]
    
        line = link_df[(link_df.tomtom_shieldnum == str(route)) & (link_df.tomtom_rtedir == direction)].copy()
        
        if len(line) == 0:
            print("\t tomtom does not label direction {}, route {}".format(direction, route))
            continue
        
        hits = bbox.apply(lambda row: list(line.sindex.intersection(row)),
                  axis = 1)
        
        tmp = pd.DataFrame({
            # index of points table
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            # ordinal position of line - access via iloc later
            "line_i": np.concatenate(hits.values)
        })
        
        # join with pems
        
        tmp.set_index(["pt_idx"], inplace = True)
        
        tmp = tmp.join(pems_subset_gdf[["station", "longitude", "latitude", "route", "direction", "geometry"]].rename(
                                columns = {"geometry" : "point"}), 
                       how = "left")
        
        # join with links
        
        tmp.set_index(["line_i"], inplace = True)
        
        tmp = tmp.join(line[["shstReferenceId", "tomtom_shieldnum", "tomtom_rtedir", "geometry"]].reset_index(drop=True), 
                       how="left")
        
        # find closest line to point
        
        tmp = gpd.GeoDataFrame(tmp, geometry = tmp["geometry"], crs = pems_gdf.crs)
        
        tmp["snap_distance"]  = tmp.geometry.distance(gpd.GeoSeries(tmp.point))
        
        tmp.sort_values(by = ["snap_distance"], inplace = True)
        
        closest = tmp.groupby(["station", "longitude", "latitude"]).first().reset_index()
        
        pems_match_gdf = pd.concat([pems_match_gdf, closest],
                                   sort = False,
                                   ignore_index = True)

pems route id 101
	 pems direction S
	 pems direction N
pems route id 80
	 pems direction W
	 pems direction E
pems route id 680
	 pems direction N
	 pems direction S
pems route id 280
	 pems direction N
	 pems direction S
pems route id 880
	 pems direction S
	 pems direction N
pems route id 580
	 pems direction W
	 pems direction E
pems route id 24
	 pems direction E
	 pems direction W
pems route id 4
	 pems direction E
	 pems direction W
pems route id 238
	 pems direction N
	 pems direction S
pems route id 85
	 pems direction N
	 pems direction S
pems route id 87
	 pems direction S
	 pems direction N
pems route id 17
	 pems direction S
	 pems direction N
pems route id 237
	 pems direction E
	 pems direction W
pems route id 242
	 pems direction N
	 pems direction S
pems route id 92
	 pems direction E
	 pems direction W
pems route id 1
	 pems direction N
	 pems direction S
pems route id 980
	 pems direction E
	 pems direction W
pems route id 25
	 pems direction S
	 tomtom does not labe

In [275]:
pems_nearest_gdf = pd.merge(pems_gdf,
                                     pems_match_gdf.drop(["point", "geometry"], axis = 1),
                                     how = "left",
                                    on = ["station", "longitude", "latitude", "route", "direction"])

In [276]:
pems_gdf.shape

(76057, 23)

In [277]:
pems_conflation_result_gdf.shape

(76057, 27)

In [278]:
pems_match_gdf[pems_match_gdf.station == 400260]

,station,longitude,latitude,route,direction,point,shstReferenceId,tomtom_shieldnum,tomtom_rtedir,geometry,snap_distance
22,400260.0,-122.393983,37.703072,101,S,POINT (-2116405.519119958 4604919.938460853),e0baa6ac9c09cf1bf3a86f61942ae2a0,101,S,LINESTRING (-2116265.063105721 4605784.1925672...,1.093781
544,400260.0,-122.394455,37.706585,101,N,POINT (-2116310.310158523 4605334.623277964),b958a1373a6c1ce3a73a6452ad1ddb96,101,N,LINESTRING (-2116435.958658942 4604533.4837011...,1.892163
545,400260.0,-122.393983,37.703072,101,N,POINT (-2116405.519119958 4604919.938460853),b958a1373a6c1ce3a73a6452ad1ddb96,101,N,LINESTRING (-2116435.958658942 4604533.4837011...,28.904669


In [279]:
pems_nearest_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 76057 entries, 0 to 76056
Data columns (total 27 columns):
station             76057 non-null int64
district            76057 non-null int64
route               76057 non-null int64
direction           76057 non-null object
type                76057 non-null object
time_period         76057 non-null object
lanes               76057 non-null int64
median_flow         76057 non-null float64
avg_flow            76057 non-null float64
sd_flow             76057 non-null float64
median_speed        60350 non-null float64
avg_speed           60350 non-null float64
sd_speed            60350 non-null float64
median_occup        76057 non-null float64
avg_occup           76057 non-null float64
sd_occupancy        76057 non-null float64
days_observed       76057 non-null int64
state_pm            76057 non-null float64
abs_pm              76057 non-null float64
latitude            76057 non-null float64
longitude           76057 non-null f

# Add shst conflation

In [283]:
pems_shst_match_df = read_shst_extract(data_interim_dir + "mtc/", "*pems.out.matched.geojson")

pems_shst_match_df.rename(columns = {"shstFromIntersectionId" : "fromIntersectionId",
                                   "shstToIntersectionId" : "toIntersectionId"},
                       inplace = True)

----------start reading shst extraction data-------------
reading shst extraction data :  ../../data/interim/mtc\car_rules\pems.out.matched.geojson
----------finished reading shst extraction data-------------


In [285]:
pems_shst_match_df.rename(columns = {"pp_station" : "station", "pp_longitude" : "longitude", "pp_latitude" : "latitude",
                                     "referenceId" : "shstReferenceId"},
                          inplace = True)

In [286]:
# if pems is not matched in previous method, use shst match

In [287]:
pems_nearest_not_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.isnull()].copy()

In [306]:
pems_nearest_not_matched_df = pems_nearest_not_matched_df.round({"longitude" : 6, "latitude" : 6})

In [307]:
pems_use_shst_result_df = pd.merge(pems_nearest_not_matched_df.drop("shstReferenceId", axis = 1),
                                   pems_shst_match_df[["shstReferenceId", 'station', 'longitude', 'latitude', 'source']],
                                   how = "left",
                                   on = ['station', 'longitude', 'latitude'])

In [308]:
pems_use_shst_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7828 entries, 0 to 7827
Data columns (total 28 columns):
station             7828 non-null int64
district            7828 non-null int64
route               7828 non-null int64
direction           7828 non-null object
type                7828 non-null object
time_period         7828 non-null object
lanes               7828 non-null int64
median_flow         7828 non-null float64
avg_flow            7828 non-null float64
sd_flow             7828 non-null float64
median_speed        6668 non-null float64
avg_speed           6668 non-null float64
sd_speed            6668 non-null float64
median_occup        7828 non-null float64
avg_occup           7828 non-null float64
sd_occupancy        7828 non-null float64
days_observed       7828 non-null int64
state_pm            7828 non-null float64
abs_pm              7828 non-null float64
latitude            7828 non-null float64
longitude           7828 non-null float64
year            

# Concat match result from two methods

In [309]:
pems_nearest_matched_df = pems_nearest_gdf[pems_nearest_gdf.shstReferenceId.notnull()].copy()

In [310]:
pems_conflation_result_df = pd.concat(
        [pems_nearest_matched_df, pems_use_shst_result_df],
        sort = False,
        ignore_index = True
)

In [311]:
pems_conflation_result_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 76057 entries, 0 to 76056
Data columns (total 28 columns):
station             76057 non-null int64
district            76057 non-null int64
route               76057 non-null int64
direction           76057 non-null object
type                76057 non-null object
time_period         76057 non-null object
lanes               76057 non-null int64
median_flow         76057 non-null float64
avg_flow            76057 non-null float64
sd_flow             76057 non-null float64
median_speed        60350 non-null float64
avg_speed           60350 non-null float64
sd_speed            60350 non-null float64
median_occup        76057 non-null float64
avg_occup           76057 non-null float64
sd_occupancy        76057 non-null float64
days_observed       76057 non-null int64
state_pm            76057 non-null float64
abs_pm              76057 non-null float64
latitude            76057 non-null float64
longitude           76057 non-null f

In [315]:
pems_conflation_result_df["source"].fillna("nearest", inplace = True)

In [316]:
pems_conflation_result_df.to_file(data_interim_dir + "mtc/pems_conflation_result.geojson",
                                  driver = "GeoJSON")